### Try to build a an agent that will work with client to compile and execute a sql order
 - bonus: build a tool to email the order details (including price) when done

In [1]:
# open ai authentication

import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
import dbio

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [22]:
# email util

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def get_mail_server(server='smtp.gmail.com', port=587):
    '''Allocate mail server on port'''
    mailServer = smtplib.SMTP(server, port)
    test = mailServer.starttls()  # start TLS
    assert test[1].lower().find(b'ready') != -1
    return mailServer

def format_message(message, sender, receiver, subject):
    '''
    Format arguments into MIMEMultipart object
    '''

    # message
    msg = MIMEMultipart()
    msg['From'] = sender
    msg['To'] = ', '.join(receiver)
    msg['Subject'] = subject
    msg.attach(MIMEText(message, 'html'))

    return msg

def send_message(msg, sender, receiver, verbose=False):
    if verbose:
        print(f'sending mail from {sender} to {receiver}')
    with get_mail_server() as mailServer:
        mailServer.sendmail(sender, receiver, msg.as_string())

In [24]:
msg = format_message('hello time', 'tweinzirl@gmail.com', 'tweinzirl@gmail.com', 'test mail')
send_message(msg, 'orderbot.com', 'tweinzirl@gmail.com', verbose=True)

#enabling gmail will require 2-factor authentication with google

sending mail from orderbot.com to tweinzirl@gmail.com


SMTPSenderRefused: (530, b'5.7.0 Authentication Required. Learn more at\n5.7.0  https://support.google.com/mail/?p=WantAuthError 15-20020a17090a004f00b0025352448ba9sm14414384pjb.0 - gsmtp', 'orderbot.com')

# Instantiate the components separately

In [ ]:
# langchain
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor

In [ ]:
# llm
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, verbose=True)

# database
db = SQLDatabase.from_uri("sqlite:///./pizza_v2.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [ ]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [ ]:
agent_executor.run('describe the pizzas table')

In [ ]:
from langchain.agents import initialize_agent

In [ ]:
initialize_agent?

# try with general chat and sql tool

 - following the example [here](https://python.langchain.com/en/latest/modules/agents/agents/examples/chat_conversation_agent.html) of an agent that can call tools and chat with the usert too
 - Note there is no prompt specific setting the context for the OrderBot

In [2]:
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [3]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [4]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, verbose=True)

In [5]:
db = SQLDatabase.from_uri("sqlite:///./pizza_v2.db")
from langchain.chains import SQLDatabaseSequentialChain
seq_chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)

In [6]:
tools = [
    Tool(
        name = "menu search",
        func=seq_chain.run,
        description="""useful for when you need to answer questions about menu items the user wants to order.
        Search your chat history before deciding you need to use this."""
    ),
]

In [7]:
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
                               verbose=True, memory=memory)
# access chat memory with agent_chain.memory

In [8]:
agent_chain.run(input="hi, i am Tim")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "Hello Tim! How can I assist you today?"
}

> Finished chain.


'Hello Tim! How can I assist you today?'

In [9]:
agent_chain.run('How many sizes of bottled water do you have?')



> Entering new AgentExecutor chain...
{
    "action": "menu search",
    "action_input": "bottled water sizes"
}

> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['drinks']

> Entering new SQLDatabaseChain chain...
bottled water sizes
SQLQuery:SELECT Price_Small FROM drinks WHERE Item = 'Bottled water'
SQLResult: [(None,)]
Answer:Bottled water comes in a single size of 20 ounces.
> Finished chain.

> Finished chain.

Observation: Bottled water comes in a single size of 20 ounces.
Thought:{
    "action": "Final Answer",
    "action_input": "Bottled water comes in a single size of 20 ounces."
}

> Finished chain.


'Bottled water comes in a single size of 20 ounces.'

In [10]:
agent_chain.run('I will take the bottled water.  What is you most popular pizza?')



> Entering new AgentExecutor chain...
```json
{
    "action": "menu search",
    "action_input": "What is your most popular pizza?"
}
```

> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['pizzas']

> Entering new SQLDatabaseChain chain...
What is your most popular pizza?
SQLQuery:SELECT Item, Description FROM pizzas ORDER BY Price_Large DESC LIMIT 1
SQLResult: [('Pepperoni pizza', 'Our most popular')]
Answer:Pepperoni pizza
> Finished chain.

> Finished chain.

Observation: Pepperoni pizza
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Our most popular pizza is pepperoni pizza."
}
```

> Finished chain.


'Our most popular pizza is pepperoni pizza.'

In [11]:
agent_chain.run(input="""I would like the large pepperoni pizza
topped with both sausage and extra cheese.
How much is this pizza?
That is it.
For pickup please""")



> Entering new AgentExecutor chain...
```json
{
    "action": "menu search",
    "action_input": "large pepperoni pizza with sausage and extra cheese"
}
```

> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['toppings', 'pizzas']

> Entering new SQLDatabaseChain chain...
large pepperoni pizza with sausage and extra cheese
SQLQuery:SELECT pizzas.Item, pizzas.Price_Large, toppings.Price
FROM pizzas, toppings
WHERE pizzas.Item = 'Pepperoni pizza' AND toppings.Item IN ('Sausage', 'Extra cheese')
SQLResult: [('Pepperoni pizza', '12.95', '2.00'), ('Pepperoni pizza', '12.95', '3.00')]
Answer:The price of a large pepperoni pizza with sausage and extra cheese is $17.95 (12.95 + 2.00 + 3.00).
> Finished chain.

> Finished chain.

Observation: The price of a large pepperoni pizza with sausage and extra cheese is $17.95 (12.95 + 2.00 + 3.00).
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The price of a large pepperoni pizza with sausage and extra chee

'The price of a large pepperoni pizza with sausage and extra cheese is $17.95 (12.95 + 2.00 + 3.00).'

In [13]:
agent_chain.run(input="Please summarize my order.")



> Entering new AgentExecutor chain...
```json
{
    "action": "menu search",
    "action_input": "Can you please summarize the order for me?"
}
```

> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['pizzas']

> Entering new SQLDatabaseChain chain...
Can you please summarize the order for me?
SQLQuery:SELECT Item, Price_Large, Price_Medium, Price_Small FROM pizzas
SQLResult: [('Pepperoni pizza', '12.95', '10', '7'), ('Cheese pizza', '10.95', '9.25', '6.50'), ('Eggplant pizza', '11.95', '9.75', '6.75')]
Answer:Here is a summary of the order:
- Pepperoni pizza: Large $12.95, Medium $10, Small $7
- Cheese pizza: Large $10.95, Medium $9.25, Small $6.50
- Eggplant pizza: Large $11.95, Medium $9.75, Small $6.75
> Finished chain.

> Finished chain.

Observation: Here is a summary of the order:
- Pepperoni pizza: Large $12.95, Medium $10, Small $7
- Cheese pizza: Large $10.95, Medium $9.25, Small $6.50
- Eggplant pizza: Large $11.95, Medium $9.75, Small $6.75
Thought:``

'Your order is a large pepperoni pizza with sausage and extra cheese for $17.95.'

In [14]:
agent_chain.run(input="How much do I owe?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Your order of a large pepperoni pizza with sausage and extra cheese costs $17.95."
}
```

> Finished chain.


'Your order of a large pepperoni pizza with sausage and extra cheese costs $17.95.'

In [ ]:
agent_chain.run(input="can you email me the receipt")

In [ ]:
initialize_agent?

Try again with custom bot following example here: https://python.langchain.com/en/latest/modules/agents/agents/custom_llm_agent.html

In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish

from langchain.sql_database import SQLDatabase
from langchain.chains import SQLDatabaseSequentialChain

from langchain.memory import ConversationBufferMemory
#from langchain.agents import initialize_agent # alternative to AgentExecutor?
#from langchain.agents import AgentType

import re

In [ ]:
# llm
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, verbose=True)

# memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# db
db = SQLDatabase.from_uri("sqlite:///./pizza_v2.db")
seq_chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)

# tools
tools = [
    Tool(
        name = "menu search",
        func=seq_chain.run,
        description="useful for when you need to answer questions about menu items the user wants to order"
    ),
]

In [ ]:
# prompt template

# Set up the base template
template_with_history = """You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \

You have access to the following tools: \

{tools} \

Use the following format: \

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [ ]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [ ]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)

In [ ]:
# Output parser
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()

In [ ]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)

In [ ]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [ ]:
agent_executor.run("How many pizzas do you have?")